In [ ]:
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods import vector
from coffea import processor
import hist
import json
import awkward as ak
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_short_name = {
    'ZJetsToNuNu_HT-100To200_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-100To200',
    'ZJetsToNuNu_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-200To400',
    'ZJetsToNuNu_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-400To600',
    'ZJetsToNuNu_HT-600To800_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-600To800',
    'ZJetsToNuNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-800To1200',
    'ZJetsToNuNu_HT-1200To2500_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-1200To2500',
    'ZJetsToNuNu_HT-2500ToInf_TuneCP5_13TeV-madgraphMLM-pythia8':'ZJetsToNuNu_HT-2500ToInf'
}

class BackgroundEstimatorProcessor(processor.ProcessorABC):
    def __init__(self):
        self.make_output = lambda: {
            'sumw':0.,
            'met_kin': hist.Hist(hist.axis.Regular(40,0,400,name='met',label='$p_T^{miss}$ [GeV]'))
        }
        
    def process(self,events):
        dataset = events.metadata['dataset']
        mask = events.MET.pt > 30
        met = events.MET[mask]

        output = self.make_output()
        output['sumw'] = ak.sum(events.genWeight)
        output['met_kin'].fill(met = met.pt)

        return {dataset_short_name[dataset]: output}

    def postprocess(self,accumulator):
        return accumulator

In [ ]:
with open("data_ZToNuNu.json","r") as f:
    fileset = json.load(f)

for dataset,val in fileset.items():
    fileset[dataset] = ["root://cmsxrootd.fnal.gov/" + file for file in val ]

out = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=BackgroundEstimatorProcessor(),
    executor=processor.iterative_executor,
    executor_args={
        "schema": NanoAODSchema
    }
)

In [ ]:
plt.figure(figsize=(9,6))
for dataset in out:
    out[dataset]["met_kin"].plot1d(density=True,label=dataset)
    plt.legend();

In [ ]:
xsecs = {
    "ZJetsToNuNu_HT-100To200": 267,
    "ZJetsToNuNu_HT-200To400": 73.08,
    "ZJetsToNuNu_HT-400To600": 13.21,
    "ZJetsToNuNu_HT-600To800": 2.413,
    "ZJetsToNuNu_HT-800To1200": 1.071,
    "ZJetsToNuNu_HT-1200To2500": 0.2497,
    "ZJetsToNuNu_HT-2500ToInf": 0.005618
}

luminosity = {
    "2016": 36330.0,
    "2017": 41480.0,
    "2018": 59830.0
}

scaled_hist = {}
for dataset,histo in out.items():
    xsec = xsecs[dataset]
    lumi = luminosity["2017"]
    sumw = histo["sumw"]
    weight = (xsec*lumi)/sumw

    h = histo.copy()
    h["met_kin"] *= weight
    scaled_hist[dataset] = h

In [ ]:
for dataset in out:
    scaled_hist[dataset]["met_kin"].plot1d(label=dataset)
    plt.legend();

In [ ]:
processor.accumulate(scaled_hist.values())["met_kin"].plot1d();